In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat May 15 10:03:22 2021

@author: arist
"""

from keras.models import load_model
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import pandas as pd
import numpy as np


model1 = load_model('/content/drive/MyDrive/COVID/COVID/CODIGOS/conjuntoImagem/inceptionCANCER.h5')
model2 = load_model('/content/drive/MyDrive/COVID/COVID/CODIGOS/conjuntoImagem/resnetCANCER.h5')
model3 = load_model('/content/drive/MyDrive/COVID/COVID/CODIGOS/conjuntoImagem/vggCANCER.h5')
model4 = load_model('/content/drive/MyDrive/COVID/COVID/CODIGOS/conjuntoImagem/xcptionocertoCANCER.h5')

models = [model1, model2, model3, model4]

CATEGORIES = ['Malignant cases', 'Normal cases']
NUM_CATEGORIES = len(CATEGORIES)
NUM_CATEGORIES

teste_dir = '/content/drive/MyDrive/COVID/COVID/CODIGOS/conjuntoImagem/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset/test'
teste = []
for category_id, category in enumerate(CATEGORIES):
    for file in os.listdir(os.path.join(teste_dir, category)):
        teste.append(['{}/{}'.format(category, file), category_id, category])
teste = pd.DataFrame(teste, columns=['file', 'category_id', 'category'])
teste.shape

test_datagen = ImageDataGenerator(rescale=1./255)
print("==============TEST RESULTS============")


test_generator = test_datagen.flow_from_dataframe(dataframe=teste,
                                                  directory=teste_dir,
                                                x_col="file",
                                                y_col='category',
                                                batch_size=10,
                                                shuffle=False,
                                                class_mode='categorical',
                                                target_size=(224, 224),
                                                validate_filenames=False)


models = [model1, model2, model3, model4]
preds1 = [model.predict(test_generator, verbose=1) for model in models]
preds1 = np.array(preds1)
true_classes = test_generator.classe


df = pd.DataFrame([])

for w1 in range(0, 5):
    for w2 in range(0,5):
        for w3 in range(0,5):
            for w4 in range(0,5):
                wts = [w1/10., w2/10., w3/10., w4/10.]
                wted_preds1 = np.tensordot(preds1, wts, axes=((0),(0)))
                wetd_ensemble_pred = np.argmax(wted_preds1, axis=1)
                weighted_accuracy1 = accuracy_score(true_classes, wetd_ensemble_pred)
                df= df.append(pd.DataFrame({'w1': wts[0], 'w2': wts[1], 'w3': wts[2], 'w4': wts[3], 'acc': weighted_accuracy1*100}, index=[0]), ignore_index=True)

                              
max_acc_row = df.iloc[df['acc'].idxmax()]
print('max acc: ', max_acc_row[0], 'w1: ', max_acc_row[1], 'w2: ', max_acc_row[2], 'w3: ', max_acc_row[3], 'w4: ', max_acc_row[4])




==============TEST RESULTS============
Found 226 non-validated image filenames belonging to 2 classes.
    137/Unknown - 240s 2s/step